In [1]:
import pandas as pd
import os
import re

In [12]:
gdp=pd.read_csv('gdp_final.csv')

In [13]:
gdp.head(5)

,GeoName,ComponentName,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,Alabama,Gross domestic product (GDP) by state,104309.0,109776.0,115704.0,120428.0,124885.0,130180.0,136270.0,148769.0,157914.0,165131.0,170399.0,173521.0,169586.0,176221.0,181923.0,187283.0,191605.0,197535.0
1,Alaska,Gross domestic product (GDP) by state,25763.0,24067.0,24727.0,26932.0,28942.0,29808.0,32125.0,35510.0,40277.0,44867.0,49285.0,55436.0,50514.0,54220.0,59318.0,61614.0,59891.0,58067.0
2,Arizona,Gross domestic product (GDP) by state,132795.0,143542.0,156118.0,166108.0,172634.0,179892.0,192663.0,204659.0,227047.0,248075.0,262214.0,258978.0,243102.0,247333.0,255621.0,266131.0,271072.0,281559.0
3,Arkansas,Gross domestic product (GDP) by state,60248.0,62695.0,67087.0,69111.0,71128.0,74243.0,78882.0,85315.0,90112.0,95727.0,99210.0,102100.0,100098.0,104929.0,109378.0,111541.0,116651.0,121065.0
4,California,Gross domestic product (GDP) by state,1082051.0,1158822.0,1258697.0,1377014.0,1394863.0,1449536.0,1535202.0,1643908.0,1760508.0,1868969.0,1951997.0,1993589.0,1913674.0,1960935.0,2031348.0,2121602.0,2215232.0,2324995.0


In [21]:
gdp.head(2)
gdp_2016=gdp.loc[:,['GeoName','ComponentName','2011','2012']]
gdp_2016['lagged']=gdp_2016['2012']-gdp_2016['2011']
gdp_2016=gdp_2016.loc[:,['GeoName','ComponentName','lagged']]
gdp_2016_cleaned=gdp_2016.pivot(index='GeoName', columns='ComponentName', values='lagged').reset_index().rename_axis(None, axis=1)

In [17]:
output_dir='./dependent_variable_state_data_cleaned/'
output_file_list=os.listdir(output_dir)

In [18]:
final_result=pd.DataFrame(columns=['state','net_change','total_expand','expand_establish',
                     'open_establish','total_contract','contract_establish','end_establish'])
year_list=[2011,2012]
for i in output_file_list:
    temp_state=re.sub('.txt','',i)
    temp_df=pd.read_table(output_dir+i,sep='/',header=None)
    temp_df.columns=['year','month','net_change','total_expand','expand_establish',
                     'open_establish','total_contract','contract_establish','end_establish']
    temp_df=temp_df.groupby('year').sum()
    temp_df=temp_df.loc[year_list].shift(-1)-temp_df.loc[year_list]
    temp_df=temp_df.dropna()
    temp_df['state']=temp_state
    final_result=final_result.append(temp_df)

In [23]:
final_y_gdp=pd.merge(final_result,gdp_2016_cleaned,left_on=final_result.state,right_on=gdp_2016_cleaned.GeoName,how='inner')

In [24]:
final_y_gdp.to_csv('final_y.csv',index=False)

In [28]:
final_y_gdp.head(2)

,contract_establish,end_establish,expand_establish,net_change,open_establish,state,total_contract,total_expand,GeoName,Compensation of employees,Gross domestic product (GDP) by state,Gross operating surplus,Per capita real GDP by state,Quantity indexes for real GDP by state,Real GDP by state,Subsidies,Taxes on production and imports,Taxes on production and imports less subsidies
0,-24042.0,-8950.0,-4220.0,27294.0,-1478.0,Alabama,-32992.0,-5698.0,Alabama,1134.0,5360.0,4004.0,247.0,1.041,1765.0,11.0,211.0,222.0
1,1978.0,3104.0,3564.0,-1740.0,-222.0,Alaska,5082.0,3342.0,Alaska,1282.0,2296.0,818.0,3056.0,5.623,2840.0,6.0,189.0,195.0


In [31]:
final_x=pd.read_csv('final_data.csv')

In [33]:
final_all=pd.merge(final_x,final_y_gdp,left_on=final_x.state,right_on=final_y_gdp.GeoName,how='inner')

In [34]:
final_all.to_csv('final_all.csv',index=False)